In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Vanishing/Exploding Gradient Problem

In [2]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [3]:
#leaky ReLU
def leaky_relu(z,alpha=0.01):
    return np.maximum(alpha*z,z)

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [4]:
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()
X_valid,X_train = X_train_full[:5000],X_train_full[5000:]
y_valid,y_train = y_train_full[:5000],y_train_full[5000:]
X_train_full = X_train_full/255
X_test = X_test/255

In [7]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300,kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100,kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10,activation="softmax")
])

In [8]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.SGD(learning_rate=1e-3),
             metrics=["accuracy"])

In [9]:
history = model.fit(X_train,y_train,epochs=10,
                   validation_data=(X_valid,y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 7s 135us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 2/10
55000/55000 [==============================] - 7s 121us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 3/10
55000/55000 [==============================] - 6s 113us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 4/10
55000/55000 [==============================] - 6s 117us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 5/10
55000/55000 [==============================] - 7s 121us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 6/10
55000/55000 [==============================] - 6s 116us/sample - loss: nan - accuracy: 0.1008 - val_loss: nan - val_accuracy: 0.0914
Epoch 7/10
55000/55000 [==============================] - 6s 118us/sample - loss: nan -

# Batch Normalization

In [10]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10,activation="softmax")
])

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_5 (Dense)              (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_6 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

In [12]:
bn1 = model.layers[1]
[(var.name,var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [13]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.SGD(learning_rate=1e-3),
             metrics=["accuracy"])

In [14]:
history = model.fit(X_train,y_train,epochs=10,
                   validation_data=(X_valid,y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 16s 288us/sample - loss: 0.8581 - accuracy: 0.7109 - val_loss: 0.6030 - val_accuracy: 0.8066
Epoch 2/10
55000/55000 [==============================] - 15s 269us/sample - loss: 0.5879 - accuracy: 0.7958 - val_loss: 0.5660 - val_accuracy: 0.8260
Epoch 3/10
55000/55000 [==============================] - 14s 264us/sample - loss: 0.5252 - accuracy: 0.8165 - val_loss: 0.5160 - val_accuracy: 0.8362
Epoch 4/10
55000/55000 [==============================] - 14s 259us/sample - loss: 0.4903 - accuracy: 0.8275 - val_loss: 0.4589 - val_accuracy: 0.8446
Epoch 5/10
55000/55000 [==============================] - 14s 256us/sample - loss: 0.4657 - accuracy: 0.8350 - val_loss: 0.4380 - val_accuracy: 0.8518
Epoch 6/10
55000/55000 [==============================] - 14s 257us/sample - loss: 0.4473 - accuracy: 0.8416 - val_loss: 0.4331 - val_accuracy: 0.8574
Epoch 7/10
55000/55000 [=====================

# Reusing Pretrained Layers

Let's split the fashion MNIST training set in two:

X_train_A: all images of all items except for sandals and shirts (classes 5 and 6).
X_train_B: a much smaller training set of just the first 200 images of sandals or shirts.
The validation set and the test set are also split this way, but without restricting the number of images.

We will train a model on set A (classification task with 8 classes), and try to reuse it to tackle set B (binary classification). We hope to transfer a little bit of knowledge from task A to task B, since classes in set A (sneakers, ankle boots, coats, t-shirts, etc.) are somewhat similar to classes in set B (sandals and shirts). However, since we are using Dense layers, only patterns that occur at the same location can be reused (in contrast, convolutional layers will transfer much better, since learned patterns can be detected anywhere on the image, as we will see in the CNN chapter).

In [25]:
def split_dataset(X,y):
    y_5_or_6 = (y == 5)|(y == 6) #sandals or shirt
    y_A = y[~y_5_or_6]
    y_A[y_A>6] -= 2 #class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6]==6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6],y_A),(X[y_5_or_6],y_B))

(X_train_A,y_train_A),(X_train_B,y_train_B) = split_dataset(X_train,y_train)
(X_valid_A,y_valid_A),(X_valid_B,y_valid_B) = split_dataset(X_valid,y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [16]:
X_train_A.shape

(43986, 28, 28)

In [17]:
X_train_B.shape

(200, 28, 28)

In [18]:
y_train_A[:20]

array([4, 0, 7, 9, 9, 9, 4, 4, 3, 4, 0, 1, 8, 3, 4, 3, 2, 8, 7, 3],
      dtype=uint8)

In [19]:
y_train_B[:20]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)

In [20]:
tf.random.set_seed(42)
np.random.seed(42)

In [29]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in (300,100,50,50,50):
    model_A.add(keras.layers.Dense(n_hidden,activation="selu"))
model_A.add(keras.layers.Dense(8,activation="softmax"))

In [30]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [31]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Train on 43986 samples, validate on 4014 samples
Epoch 1/20
43986/43986 [==============================] - 12s 280us/sample - loss: 1.2488 - accuracy: 0.8183 - val_loss: 0.4112 - val_accuracy: 0.8513
Epoch 2/20
43986/43986 [==============================] - 12s 263us/sample - loss: 0.3449 - accuracy: 0.8780 - val_loss: 0.3601 - val_accuracy: 0.8705
Epoch 3/20
43986/43986 [==============================] - 11s 253us/sample - loss: 0.3001 - accuracy: 0.8936 - val_loss: 0.3027 - val_accuracy: 0.8931
Epoch 4/20
43986/43986 [==============================] - 11s 257us/sample - loss: 0.2750 - accuracy: 0.9026 - val_loss: 0.2832 - val_accuracy: 0.9038
Epoch 5/20
43986/43986 [==============================] - 11s 258us/sample - loss: 0.2576 - accuracy: 0.9097 - val_loss: 0.3023 - val_accuracy: 0.8984
Epoch 6/20
43986/43986 [==============================] - 11s 251us/sample - loss: 0.2428 - accuracy: 0.9147 - val_loss: 0.2728 - val_accuracy: 0.9078
Epoch 7/20
43986/43986 [=====================

In [32]:
model_A.save("my_model_A.h5")

In [33]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28,28]))
for n_hidden in(300,100,50,50,50):
    model_B.add(keras.layers.Dense(n_hidden,activation="selu"))
model_B.add(keras.layers.Dense(1,activation="sigmoid"))

In [34]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [35]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/20
200/200 [==============================] - 1s 6ms/sample - loss: 18.2954 - accuracy: 0.6550 - val_loss: 0.4970 - val_accuracy: 0.9361
Epoch 2/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.0883 - accuracy: 0.9750 - val_loss: 0.5204 - val_accuracy: 0.9462
Epoch 3/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.0331 - accuracy: 0.9850 - val_loss: 0.4483 - val_accuracy: 0.9554
Epoch 4/20
200/200 [==============================] - 0s 1ms/sample - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.4260 - val_accuracy: 0.9574
Epoch 5/20
200/200 [==============================] - 0s 971us/sample - loss: 6.9623e-04 - accuracy: 1.0000 - val_loss: 0.4180 - val_accuracy: 0.9584
Epoch 6/20
200/200 [==============================] - 0s 987us/sample - loss: 4.7224e-04 - accuracy: 1.0000 - val_loss: 0.4133 - val_accuracy: 0.9594
Epoch 7/20
200/200 [==============================] - 0s 996us/sample - loss:

In [36]:
model_B.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 300)               235500    
_________________________________________________________________
dense_27 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_28 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_29 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_30 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_31 (Dense)             (None, 1)                

In [37]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1,activation="sigmoid"))

In [38]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [41]:
for layers in model_B_on_A.layers[:-1]:
    layers.trainable = False
    
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [42]:
history = model_B_on_A.fit(X_train_B,y_train_B,epochs=4,
                          validation_data=(X_valid_B,y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/4
200/200 [==============================] - 1s 5ms/sample - loss: 2.8058 - accuracy: 0.3450 - val_loss: 1.9928 - val_accuracy: 0.4361
Epoch 2/4
200/200 [==============================] - 0s 923us/sample - loss: 1.7153 - accuracy: 0.4650 - val_loss: 1.2161 - val_accuracy: 0.5609
Epoch 3/4
200/200 [==============================] - 0s 915us/sample - loss: 1.0661 - accuracy: 0.6000 - val_loss: 0.7833 - val_accuracy: 0.6755
Epoch 4/4
200/200 [==============================] - 0s 890us/sample - loss: 0.7073 - accuracy: 0.6750 - val_loss: 0.5117 - val_accuracy: 0.7677


In [43]:
for layers in model_B_on_A.layers[:-1]:
    layers.trainable = True
    
model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B,y_train_B,epochs=4,
                          validation_data=(X_valid_B,y_valid_B))

Train on 200 samples, validate on 986 samples
Epoch 1/4
200/200 [==============================] - 1s 5ms/sample - loss: 0.2682 - accuracy: 0.8650 - val_loss: 0.1198 - val_accuracy: 0.9645
Epoch 2/4
200/200 [==============================] - 0s 1ms/sample - loss: 0.0518 - accuracy: 0.9800 - val_loss: 0.0935 - val_accuracy: 0.9716
Epoch 3/4
200/200 [==============================] - 0s 1ms/sample - loss: 0.0273 - accuracy: 1.0000 - val_loss: 0.0842 - val_accuracy: 0.9787
Epoch 4/4
200/200 [==============================] - 0s 1ms/sample - loss: 0.0181 - accuracy: 1.0000 - val_loss: 0.0777 - val_accuracy: 0.9797


In [44]:
model_B.evaluate(X_test_B,y_test_B)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.5194456725120544, 0.7195]

In [45]:
model_B_on_A.evaluate(X_test_B, y_test_B)

2000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.534353558063507, 0.685]

# Faster Optimizer

In [46]:
#Adam
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [47]:
#nadam
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

# Avoiding overfitting through Regularization

In [50]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 18s 334us/sample - loss: 8.3647 - accuracy: 0.7047 - val_loss: 4.6474 - val_accuracy: 0.7916
Epoch 2/2
55000/55000 [==============================] - 18s 323us/sample - loss: 3.3425 - accuracy: 0.7994 - val_loss: 2.2208 - val_accuracy: 0.8314


In [51]:
#dropout
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/2
55000/55000 [==============================] - 21s 374us/sample - loss: 4.3468 - accuracy: 0.4001 - val_loss: 1.1287 - val_accuracy: 0.5420
Epoch 2/2
55000/55000 [==============================] - 19s 340us/sample - loss: 1.3200 - accuracy: 0.5091 - val_loss: 0.8454 - val_accuracy: 0.6774


# Exercise

8. a.Build a DNN with 20 hidden layers of 100 neurons each (that's too many, but it's the point of this exercise). Use He initialization and the ELU activation function.

In [52]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

b. Exercise: Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with keras.datasets.cifar10.load_data(). The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you'll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model's architecture or hyperparameters

In [54]:
(X_train_full,y_train_full),(X_test,y_test) = keras.datasets.cifar10.load_data()

In [67]:
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [68]:
X_train.shape

(45000, 32, 32, 3)

In [69]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
for n_hidden in range(20):
    model.add(keras.layers.Dense(100,activation="elu",
                                kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [70]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
               optimizer=optimizer,
               metrics=["accuracy"])

In [71]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
callbacks = [early_stopping_cb]

In [72]:
model.fit(X_train,y_train,epochs=50,
         validation_data=(X_valid,y_valid),callbacks = callbacks)

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 49s 1ms/sample - loss: 5.9475 - accuracy: 0.1411 - val_loss: 2.1827 - val_accuracy: 0.2006
Epoch 2/50
45000/45000 [==============================] - 45s 990us/sample - loss: 2.0703 - accuracy: 0.2334 - val_loss: 1.9917 - val_accuracy: 0.2626
Epoch 3/50
45000/45000 [==============================] - 45s 1ms/sample - loss: 1.9402 - accuracy: 0.2786 - val_loss: 1.9036 - val_accuracy: 0.2842
Epoch 4/50
45000/45000 [==============================] - 45s 1ms/sample - loss: 1.8728 - accuracy: 0.3109 - val_loss: 1.8393 - val_accuracy: 0.3178
Epoch 5/50
45000/45000 [==============================] - 44s 981us/sample - loss: 1.8137 - accuracy: 0.3355 - val_loss: 1.8020 - val_accuracy: 0.3498
Epoch 6/50
45000/45000 [==============================] - 45s 994us/sample - loss: 1.7600 - accuracy: 0.3575 - val_loss: 1.7153 - val_accuracy: 0.3774
Epoch 7/50
45000/45000 [===========================

In [76]:
model.evaluate(X_valid, y_valid)

5000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[1.5636629913330078, 0.4626]

c. Exercise: Now try adding Batch Normalization and compare the learning curves: Is it converging faster than before? Does it produce a better model? How does it affect training speed?

In [78]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32,32,3]))
model.add(keras.layers.BatchNormalization())
for n_hidden in range(20):
    model.add(keras.layers.Dense(100,
                                kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10,activation="softmax"))

In [79]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [81]:
model.fit(X_train, y_train, epochs=50,
          validation_data=(X_valid, y_valid),
          )

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 76s 2ms/sample - loss: 1.7208 - accuracy: 0.3846 - val_loss: 1.6088 - val_accuracy: 0.4284
Epoch 2/50
45000/45000 [==============================] - 76s 2ms/sample - loss: 1.6266 - accuracy: 0.4248 - val_loss: 1.5671 - val_accuracy: 0.4404
Epoch 3/50
45000/45000 [==============================] - 76s 2ms/sample - loss: 1.5650 - accuracy: 0.4441 - val_loss: 1.5088 - val_accuracy: 0.4678
Epoch 4/50
45000/45000 [==============================] - 78s 2ms/sample - loss: 1.5281 - accuracy: 0.4593 - val_loss: 1.4855 - val_accuracy: 0.4642
Epoch 5/50
45000/45000 [==============================] - 78s 2ms/sample - loss: 1.4813 - accuracy: 0.4748 - val_loss: 1.4457 - val_accuracy: 0.4800
Epoch 6/50
45000/45000 [==============================] - 78s 2ms/sample - loss: 1.4433 - accuracy: 0.4853 - val_loss: 1.4083 - val_accuracy: 0.4922
Epoch 7/50
45000/45000 [==============================] -